In [ ]:
# President: 2016 (Trump), 2020 (Biden), 2024 (Trump)
# Governor: 2018 (Whitmer), 2022 (Whitmer)
# Secretary of State: 2018 (Benson), 2022 (Benson)
# Attorney General: 2018 (Nessel), 2022 (Nessel)
# U.S. Senate: 2014 (Peters), 2018 (Stabenow), 2020 (Peters), 2024 (Slotkin)
# U.S. House: every cycle
# State Senate: 2014, 2018, 2022
# State House: every cycle

OFFICES = ['U.S. House']
# OFFICES = ['U.S. House', 'State House']
YEARS = ['2014', '2016', '2018', '2020', '2022', '2024']

# OFFICES = ['U.S. Senate']
# YEARS = ['2014', '2018', '2020', '2024']

# OFFICES = ['State Senate']
# YEARS = ['2014', '2018', '2022']

# OFFICES = ['President']
# YEARS = ['2016', '2020', '2024']

# Not enough data
# # OFFICES = ['Governor', 'Secretary of State', 'Attorney General']
# # YEARS = ['2018', '2022']

If the year is 2024, use 2023 data since 2024 data is not yet available. That is why reading files with years uses this <code>str(YEAR if YEAR != 2024 else 2023)</code> so that 2024 year reads filenames with "2023".

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.set_option("display.max_columns", None)

### Join tract data to precinct results

In [ ]:
for year in YEARS:
    for office in OFFICES:
        df_precinct_results_mapped = gpd.read_file('data/generated_data/df_05_precinct_mapped_merged_' + year + '_' + office.replace('.', '').replace(' ', '_') + '.geojson', driver='GeoJSON')
        df_precinct_results_mapped['nearest_bound_census_tract'] = df_precinct_results_mapped['nearest_bound_census_tract'].apply(lambda x: str(x)[:-2]) # Cast as string and remove the decimal
        
        # If year is 2024, use 2023 data since 2024 has no data yet.
        if year == '2024':
            census_year = '2023'
        else:
            census_year = year
        
        df_census_tracts = gpd.read_file('data/census/tracts/cb_' + census_year + '_26_tract_500k/cb_' + census_year + '_26_tract_500k.shp')
        
        df_census_tracts.columns = df_census_tracts.columns.str.lower()
        df_census_tracts['geoid'] = df_census_tracts['geoid'].astype(str)
        
        if int(year) >= 2023:
            # IF YEAR >= 2023 geoidfq
            df_census_tracts.rename(columns={'tractce': 'tractce_tract', 'geoid': 'geoid_tract', 'geoidfq': 'geoidfq_tract', 'name': 'name_tract', 'aland': 'aland_tract', 'awater': 'awater_tract', 'geometry': 'geometry_tract'}, inplace=True)
            df_census_tracts = df_census_tracts[['tractce_tract', 'geoid_tract', 'geoidfq_tract', 'name_tract', 'aland_tract', 'awater_tract', 'geometry_tract']]
        else:
            # IF YEAR < 2023 affgeoid
            df_census_tracts.rename(columns={'tractce': 'tractce_tract', 'geoid': 'geoid_tract', 'affgeoid': 'geoidfq_tract', 'name': 'name_tract', 'aland': 'aland_tract', 'awater': 'awater_tract', 'geometry': 'geometry_tract'}, inplace=True)
            df_census_tracts = df_census_tracts[['tractce_tract', 'geoid_tract', 'geoidfq_tract', 'name_tract', 'aland_tract', 'awater_tract', 'geometry_tract']]

        df_precinct_results_mapped['nearest_bound_census_tract'] = df_precinct_results_mapped['nearest_bound_census_tract'].str.ljust(11, '0') # add trailing zeros if less than 11 characters.
        df_precinct_results_tracts = pd.merge(df_precinct_results_mapped, df_census_tracts, left_on='nearest_bound_census_tract', right_on='geoid_tract', how='left')
        df_precinct_results_tracts['standardized_id_num'] = df_precinct_results_tracts['standardized_id_num'].astype(str).str.zfill(13)
        
        df_precinct_results_tracts.to_csv('data/generated_data/df_06_tract_' + year + '_' + office.replace('.', '').replace(' ', '_') + '.csv', index=False)

### Join census demographic data

In [ ]:
def makeCensusData(datasets):
    for dataset in datasets:
        for year in YEARS:
            for office in OFFICES:
                # Census datasets have varied codes
                dataset = dataset.lower()
                if dataset[:1] == 's':
                    dataset_code = dataset[:5].upper()
                    dataset_label = dataset[6:]
                    data_type = 'ACSST5Y'
                elif dataset[:1] == 'b':
                    dataset_code = dataset[:6].upper()
                    dataset_label = dataset[7:]
                    data_type = 'ACSDT5Y'
                elif dataset[:1] == 'd':
                    dataset_code = dataset[:4].upper()
                    dataset_label = dataset[5:]
                    data_type = 'ACSDP5Y'

                # If year is 2024, use 2023 data since 2024 has no data yet.
                if year == '2024':
                    census_year = '2023'
                else:
                    census_year = year
                
                df = pd.read_csv(f'data/census/{dataset}/{data_type}' + census_year + f'.{dataset_code}-Data.csv', header=0, skiprows=[1])
        
                df_columns = pd.read_csv(f'data/census/{dataset}/{data_type}' + census_year + f'.{dataset_code}-Column-Metadata.csv', header=0, skiprows=[2])
                df_columns = df_columns[~df_columns["Label"].str.contains("Margin of Error!!", na=False, case=False)]
                columns = list(df_columns['Column Name'])
                
                df = df[columns]
                df.rename(columns={'GEO_ID': f'geoid_{dataset_label}'}, inplace=True)
                df.to_csv(f'data/generated_data/df_06_{dataset_label}_' + year + '_' + office.replace('.', '').replace(' ', '_') + '.csv', index=False)
                
                df_merged = pd.merge(df_precinct_results_tracts, df, left_on='geoidfq_tract', right_on=f'geoid_{dataset_label}', how='left')
                df_merged.drop(columns=[f"geoid_{dataset_label}"], inplace=True)
                
                df_merged.to_csv(f'data/generated_data/df_06_tract_{dataset_label}_' + year + '_' + office.replace('.', '').replace(' ', '_') + '.csv', index=False)

In [ ]:
census_datasets = [
    'b02001_race', 'b04007_ancestry', 'b05012_nativity_us', 'b08303_travel_time_work', 'b25003_housing_rentership', 
    'dp02_selected_social_characteristics', 'dp03_selected_economic_characteristics', 'dp04_housing_characteristics', 'dp05_age_race', 
    's0101_age_sex', 's1101_households_families', 's1201_marital_status', 's1501_educational_attainment', 's1701_income_poverty', 
    's1903_median_income', 's2101_veteran_status', 's2201_food_stamps', 's2301_employment_status', 's2401_occupation_sex', 
    's2403_industry_sex', 's2501_occupancy_characteristics', 's2701_health_insurance', 's2503_financial_characteristics',
]

makeCensusData(census_datasets)